# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [2]:
#Import the libraries
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0,373450,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     865 non-null    float64
 2   Pclass       828 non-null    float64
 3   Name         866 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        858 non-null    float64
 7   Parch        891 non-null    int64  
 8   Ticket       852 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     803 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 83.7+ KB


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [6]:
df.shape

(891, 12)

# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)

<font color='blue'>Q2) How many columns do we have left?</font>




In [7]:
halfCount = df.shape[0]/2
df.dropna(axis = "columns", thresh = halfCount, inplace = True)
flatStandard = df.shape[0]*0.7
flatCol = []
colList = df.columns.tolist()
for col in colList :
  if df[col].value_counts().max() > flatStandard:
    flatCol.append(col)
df.drop(flatCol, axis = 1,inplace = True)
print(df.shape)


(891, 10)


# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

In [8]:
df.dropna(subset=['Survived'], axis = 0, inplace = True)
df.shape

(865, 10)

# 4) Handle outliers 

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



In [9]:
quantile =  np.percentile(df["Fare"], [25, 75])
q1 = quantile[0]
q3 = quantile[1]
iqr = q3-q1
first_con = q1 - 1.5*iqr
second_con = q3 + 1.5*iqr
df['Fare'].mask(df['Fare'] < first_con, first_con, inplace=True)
df['Fare'].mask(df['Fare'] > second_con, second_con, inplace=True)
print(df['Fare'].mean())

24.038996878612718


# 5) Impute missing value



*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>



In [10]:
print("---------before-------")
print(df.isnull().sum())
print("---------after--------")
# Select numeric columns.
a = df.select_dtypes('number')
# Select string and object columns.
b = df.select_dtypes('object')

# Fill numeric columns with mean.
df[a.columns] = a.fillna(a.mean())
# Fill object columns with mode.
df[b.columns] = b.fillna(b.agg(lambda x: x.mode().values[0]))
print(df.isnull().sum())

---------before-------
PassengerId      0
Survived         0
Pclass          63
Name            25
Sex              0
Age            172
SibSp           33
Ticket          39
Fare             0
Embarked        87
dtype: int64
---------after--------
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the dummy coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>



In [11]:
dummy_sex = pd.get_dummies(df["Sex"]).drop("female", axis = 1)
dummy_embarked = pd.get_dummies(df["Embarked"]).drop("C", axis = 1)
df = df.join(dummy_sex)
df = df.join(dummy_embarked)
df.drop(["Sex", "Embarked", "Name", "Ticket"], axis = 1, inplace = True)
df.rename(columns = {'male':'Is_male', 'Q':'Embarked_Q', 'S':'Embarked_S'}, inplace = True)
df

,PassengerId,Survived,Pclass,Age,SibSp,Fare,Is_male,Embarked_Q,Embarked_S
0,1,0.0,3.0,22.000000,1.0,7.2500,1,0,1
1,2,1.0,1.0,38.000000,1.0,65.6563,0,0,0
2,3,1.0,3.0,26.000000,0.0,7.9250,0,0,1
3,4,1.0,1.0,35.000000,1.0,53.1000,0,0,1
4,5,0.0,3.0,35.000000,0.0,8.0500,1,0,1
...,...,...,...,...,...,...,...,...,...
886,887,0.0,2.0,27.000000,0.0,13.0000,1,0,1
887,888,1.0,1.0,19.000000,0.0,30.0000,0,0,1
888,889,0.0,3.0,29.892266,1.0,23.4500,0,0,1
889,890,1.0,1.0,26.000000,0.0,30.0000,1,0,0


# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



In [14]:
from sklearn.model_selection import train_test_split
input_data = df.drop("Survived", axis = 1).values.tolist()
output_data = df["Survived"].values.tolist()
# print(input_data)
# print(output_data)
x_train, x_test, y_train, y_test = train_test_split(input_data, output_data, stratify = output_data, test_size=0.30, random_state = 12345)
print("all_row :", len(input_data))
print("all_survived :", sum(output_data))
print("all_dead :", len(input_data) - sum(output_data))
print("----------------------")
print("train_row :", len(x_train))
print("train_survived :", sum(y_train))
print("train_dead :", len(x_train) - sum(y_train))
print("----------------------")
print("test_row :", len(x_test))
print("test_survived :", sum(y_test))
print("test_dead :", len(x_test) - sum(y_test))
# from sklearn.model_selection import train_test_split 
# train, test = train_test_split(df, test_size=0.30, stratify=df['Survived'], random_state = 12345) 
# print("all_survived :", df["Survived"].sum())
# print("train_survived :", train["Survived"].sum())
# print("test_survived :", test["Survived"].sum())

all_row : 865
all_survived : 332.0
all_dead : 533.0
----------------------
train_row : 605
train_survived : 232.0
train_dead : 373.0
----------------------
test_row : 260
test_survived : 100.0
test_dead : 160.0
